<a href="https://colab.research.google.com/github/ole12345/yolov9/blob/main/yolo_v9train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ole12345/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 576, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 576 (delta 35), reused 51 (delta 21), pack-reused 499 (from 1)
Receiving objects: 100% (576/576), 2.64 MiB | 14.63 MiB/s, done.
Resolving deltas: 100% (310/310), done.


In [ ]:
%cd yolov9
!pip install -r requirements.txt
import os
os.kill(os.getpid(), 9)


/content/yolov9
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/

In [ ]:
%cd yolov9
!pwd

In [ ]:
!pip install --upgrade jax

In [ ]:
import os
from utils.filter_dataset import DatasetConfig, SubsetStrEnum, filter_dataset
import yaml

root_dir = os.getcwd()
dataset_name="coco-2017-person-ball-small"
category_list=["person","sports ball"]
yaml_content = {"path":os.path.join(".","datasets",dataset_name)}

#validation
subset = SubsetStrEnum.VALIDATION
max_number_of_images = None
cfg= DatasetConfig(root_dir = root_dir,dataset_name=dataset_name, subset = subset, max_number_of_images = max_number_of_images,category_list=category_list)
cat_id_name_val = filter_dataset(cfg)
yaml_content["val"] = subset+".txt"
#train
subset = SubsetStrEnum.TRAIN
max_number_of_images = None
cfg= DatasetConfig(root_dir = root_dir,dataset_name=dataset_name, subset = subset, max_number_of_images = max_number_of_images,category_list=category_list)
cat_id_name_train = filter_dataset(cfg)
yaml_content["train"] = subset+".txt"
if cat_id_name_train!=cat_id_name_val:
   raise Exception("Inconsistent categories")

#write the yaml file
yaml_content["names"] = cat_id_name_train
yaml_content["stuff_names"] = list()
yaml_content["download"] = None
with open(os.path.join(root_dir,"data",dataset_name+".yaml"),"w") as file:
   yaml.dump(yaml_content, file, default_flow_style=False)
   file.close()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from segment.train import run

run(data = "coco-2017-person-ball-small.yaml",
    device = 0,
    batch = 32,
    img = 640,
    cfg = "models/segment/gelan-c-seg-c.yaml",
    weights = "",
    name = "small",
    hyp = "hyp.scratch-high.yaml",
    close_mosaic = 10,
    epochs = 10,
    no_overlap = True,
    project = "/content/drive/My Drive"
)